In [ ]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
#model and 
from utils import *
from tqdm import tqdm
from models import MSE,MDcR,DSE,MVBG,CPCA,DPCA,MVP,LPP,LE
import scipy
datasets_names = ['BBC','MSRC-v1','NGs','Reuters','YALE']
datasets_names = ['Reuters','YALE']

for dt_name in datasets_names:
    # data
    X,labels=read_data(f'datasets/data sets/{dt_name}.mat')
    X = [X[0,i].T for  i  in range(X.shape[1])]
    X =  [x[:300,:] for x in X]

    if isinstance(X[0],scipy.sparse._csr.csr_matrix):
        X =   [np.asarray(x.todense()) for x in X]
    labels  = labels.squeeze()
    k = len(set(labels))
    #models
    model_MSE = MSE()
    model_MDcR = MDcR()
    model_DSE = DSE()
    model_MVBG = MVBG(0.1,2,0.1)
    model_CPCA = CPCA()
    model_DPCA = DPCA()
    model_MVP =  MVP()
    model_LPP   =   LPP()
    model_LE = LE()
    #main 
    res_nmi = []
    d_max  =   int(1*np.min([x.shape[0] for x in X]))
    d_range  = range(k,d_max)
    model = model_DPCA

    for d_ in tqdm(d_range): 
        nmi_list = []
        for i in range(5):
            # train_test split
            train_idx,test_idx = random_index(X[0].shape[1],0.8)
            X_train = [x[:,train_idx] for x in X]
            X_test = [x[:,test_idx] for x in X]
            y_test = labels[test_idx]
            #train

            # pred =  model.predict(X_test,0.5,2,1e6,d_,k,10) #MVP
            # pred   = model.predict(X_train,X_test,1e7,d_,k,200) #lPP ok too slow
            # pred  = model.predict(X_train,X_test,d_,20,k) #LE ok
            pred = model.predict(X_train,X_test,d_,k)

            # criterion
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi = normalized_mutual_info_score(y_test,pred)
            nmi_list.append(nmi)
        nmi = np.mean(nmi_list)
        res_nmi.append(nmi)
    np.save(f'./result/{dt_name}/nmi_{model.name}.npy',res_nmi)
    #plot
    import matplotlib.pyplot as plt
    plt.figure(figsize=(3,2))
    plt.plot(d_range,res_nmi)


In [5]:
from scipy.optimize import linear_sum_assignment
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import accuracy_score
def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    ind = linear_sum_assignment(w.max() - w)
    ind = np.array(ind).T
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

import numpy as np
a = np.array([1,1,2,2,3,3])
b = np.array([3,3,1,2,3,1])
# b  = np.array([1,1,3,2,1,3])

adjusted_rand_score(a,b)
accuracy_score(a,b)
acc(a,b)

0.6666666666666666

In [ ]:
from utils import *
X,labels=read_data(f'datasets/data sets/Reuters.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
np.asarray(X[2].todense()).shape

In [ ]:
# np.save('./result/NGs/mvbg_nmi.npy',res_nim)
res_nmi = np.load('./result/YALE/nmi_DPCA.npy')
res_nmi


In [ ]:
#MVBG
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
#model and 
from utils import *
from tqdm import tqdm
from models import MSE,MDcR,DSE,MVBG,PCA
from sklearn.metrics.cluster import normalized_mutual_info_score
X,labels=read_data('datasets/data sets/NGs.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()

d_range = (10,12)
model_mvbg = MVBG(0.1,2,0.1)
res_nmi = []
for d_ in d_range:
    nmi_list=[]
    for _ in range(1):
        dim_emb = model_mvbg.mvbg(X,100,d_,10)
        pred =kmeans(dim_emb,7)
        nmi_temp = normalized_mutual_info_score(labels, pred)
        nmi_list.append(nmi_temp)
    nmi = np.mean(nmi_list)
    res_nmi.append(nmi)
print(res_nmi)

In [ ]:
#DSE 單獨
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from utils import *
from tqdm import tqdm
X,labels=read_data('datasets/data sets/YALE.mat')
X = [X[0,i].T for  i  in range(X.shape[1])]
labels  = labels.squeeze()

from models import DSE
model_DSE = DSE()
nmi_list=[]

for _ in tqdm(range(20)):

    pred = model_DSE.dse(X,7,2,500)
    from sklearn.metrics.cluster import normalized_mutual_info_score
    nmi = normalized_mutual_info_score(labels,pred)
    # print(nmi)
    nmi_list.append(nmi)
print(np.mean(nmi_list))

In [ ]:
labels.shape

In [ ]:
x=np.random.rand(3,4)
np.exp(x)/np.exp(x).sum(1).reshape(3,-1)